# Plot Map
@TODO: description

## Import section

In [1]:
# standard modules
import sys
import pandas as pd

# map plotting modules
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# gis modules
import ogr
import osr
import geopandas as gpd

# my modules
sys.path.append('../code/')
import utils as mutils
import mycolors

## Prepare data

### Monthly precipitation data stations

In [2]:
# specify path to station meta file
path = '../raw_data/station_data/monthly_precip_stations.csv'
stations = pd.read_csv(path, index_col=0)
stations

,Name,Parameter,Datenquelle,Länge/Breite,Koordinaten [km],Höhe ü. M. [m]
stn,,,,,,
EIG,Eigergletscher,rre150m0,MeteoSchweiz,7°58'/46°35',640950/158350,2320
KSE,Kleine Scheidegg,rre150m0,MeteoSchweiz,7°58'/46°35',639896/159346,2060


In [3]:
# subset for neccessary information only
precip = stations[['Name', 'Koordinaten [km]', 'Höhe ü. M. [m]']].copy()
precip.columns = ['name', 'coords', 'alti']

In [4]:
# get longitude and latitude parameters
coords = precip.coords.str.split('/')

# specify CH1903+/LV95 EPSG code
EPSG = 2056  

# iterate over all stations
for stn, _ in precip.iterrows():
    # split into lon and lat value and
    # add omitted leading numbers to CH1903+ coords
    lon = float(coords[stn][0]) + 2e6
    lat = float(coords[stn][1]) + 1e6
        # convert into WGS84 coords
    lon, lat = mutils.convert_to_wgs84(lon, lat, EPSG)
    # add to DataFrame
    precip.loc[stn, 'lon'] = lon
    precip.loc[stn, 'lat'] = lat


In [5]:
precip

,name,coords,alti,lon,lat
stn,,,,,
EIG,Eigergletscher,640950/158350,2320,7.972827,46.575174
KSE,Kleine Scheidegg,639896/159346,2060,7.959164,46.584197


### SLF precipitation stations

In [6]:
# specify path to station meta file
path = '../raw_data/station_data/slf_stations.csv'
stations = pd.read_csv(path, index_col=0)
stations

,Name,Parameter,Data source,Longitude/Latitude,Coordinates [km],Elevation [m]
SLFFIR,Grindelwald / First,rre150d0,Eidg. Institut für Schnee- und Lawinenforschung,8°04'/46°40',647900/168780,2110
SLFGAD,Gadmen / Gschletteregg,rre150d0,Eidg. Institut für Schnee- und Lawinenforschung,8°24'/46°45',673270/177465,2060
SLFGO3,Goms / Treichbode,rre150d0,Eidg. Institut für Schnee- und Lawinenforschung,8°14'/46°29',660650/148950,2430
SLFGU2,Guttannen / Homad,rre150d0,Eidg. Institut für Schnee- und Lawinenforschung,8°17'/46°41',665100/170100,2110
SLFSH2,Schilthorn / Türliboden,rre150d0,Eidg. Institut für Schnee- und Lawinenforschung,7°50'/46°35',630380/158450,2360
SLFUR2,Urseren / Giltnasen,rre150d0,Eidg. Institut für Schnee- und Lawinenforschung,8°31'/46°35',682400/160100,2170


In [7]:
# subset for neccessary information only
slf = stations[['Name', 'Coordinates [km]', 'Elevation [m]']].copy()
slf.columns = ['name', 'coords', 'alti']

In [8]:
# get longitude and latitude parameters
coords = slf.coords.str.split('/')

# specify CH1903+/LV95 EPSG code
EPSG = 2056  

# iterate over all stations
for stn, _ in slf.iterrows():
    # split into lon and lat value and
    # add omitted leading numbers to CH1903+ coords
    lon = float(coords[stn][0]) + 2e6
    lat = float(coords[stn][1]) + 1e6
        # convert into WGS84 coords
    lon, lat = mutils.convert_to_wgs84(lon, lat, EPSG)
    # add to DataFrame
    slf.loc[stn, 'lon'] = lon
    slf.loc[stn, 'lat'] = lat

In [9]:
slf

,name,coords,alti,lon,lat
SLFFIR,Grindelwald / First,647900/168780,2110,8.064570,46.668533
SLFGAD,Gadmen / Gschletteregg,673270/177465,2060,8.397452,46.744351
SLFGO3,Goms / Treichbode,660650/148950,2430,8.228563,46.489124
SLFGU2,Guttannen / Homad,665100/170100,2110,8.289503,46.678951
SLFSH2,Schilthorn / Türliboden,630380/158450,2360,7.834952,46.576636
SLFUR2,Urseren / Giltnasen,682400/160100,2170,8.513800,46.587093


In [10]:
# read average annual precipitation of SLF stations
path = '../raw_data/station_data/yearly_precip_avg_slf.csv'
precip_avg_slf = pd.read_csv(path, index_col=0, squeeze=True, names=['yearly_precip'])
# add to SLF DataFrame
slf['yearly_precip'] = precip_avg_slf
slf

,name,coords,alti,lon,lat,yearly_precip
SLFFIR,Grindelwald / First,647900/168780,2110,8.064570,46.668533,1166.133356
SLFGAD,Gadmen / Gschletteregg,673270/177465,2060,8.397452,46.744351,1242.102748
SLFGO3,Goms / Treichbode,660650/148950,2430,8.228563,46.489124,802.046200
SLFGU2,Guttannen / Homad,665100/170100,2110,8.289503,46.678951,941.062073
SLFSH2,Schilthorn / Türliboden,630380/158450,2360,7.834952,46.576636,1135.776210
SLFUR2,Urseren / Giltnasen,682400/160100,2170,8.513800,46.587093,913.500802


### Temperature station

In [11]:
# specify path to station meta file
path = '../raw_data/station_data/monthly_temp_stations.csv'
stations = pd.read_csv(path, index_col=0)
stations

,Name,Parameter,Data source,Longitude/Latitude,Coordinates [km],Elevation [m]
stn,,,,,,
JUN,Jungfraujoch,tre200m0,MeteoSchweiz,7°59'/46°33',641930/155275,3580


In [12]:
# subset for neccessary information only
temp = stations[['Name', 'Coordinates [km]', 'Elevation [m]']].copy()
temp.columns = ['name', 'coords', 'alti']

In [13]:
# get longitude and latitude parameters
coords = temp.coords.str.split('/')

# specify CH1903+/LV95 EPSG code
EPSG = 2056  

# iterate over all stations
for stn, _ in temp.iterrows():
    # split into lon and lat value and
    # add omitted leading numbers to CH1903+ coords
    lon = float(coords[stn][0]) + 2e6
    lat = float(coords[stn][1]) + 1e6
    # convert into WGS84 coords
    lon, lat = mutils.convert_to_wgs84(lon, lat, EPSG)
    # add to DataFrame
    temp.loc[stn, 'lon'] = lon
    temp.loc[stn, 'lat'] = lat

In [14]:
temp

,name,coords,alti,lon,lat
stn,,,,,
JUN,Jungfraujoch,641930/155275,3580,7.985331,46.547452


### Glacier and Histalp reference location
Specify glacier and histalp reference location in same format as the stations above...

In [15]:
# define glacier DataFrame
glacier = pd.DataFrame(
    {'name': 'Upper Grindelwald Glacier',
     'lon': 8.11933,
     'lat': 46.6143}, index=['UGG'])
glacier

,name,lon,lat
UGG,Upper Grindelwald Glacier,8.11933,46.6143


In [16]:
# define HistAlp DataFrame
histalp = pd.DataFrame(
    {'name': 'HistAlp reference location',
     'lon': 8.08333333333175,
     'lat': 46.58333333333195}, index=['HISTALP'])
histalp

,name,lon,lat
HISTALP,HistAlp reference location,8.083333,46.583333


## Plot map

In [18]:
# initialize plotly in the browser
init_notebook_mode(connected=True)
# specify mapbox access toke
mapbox_access_token = 'pk.eyJ1Ijoib2JlcnJhdWNoIiwiYSI6ImNqbTBjaGprYjI3amIzdm9icDQ2bGV6NGEifQ.FmnRlRom8kc4D15utRiD_A'

# create markers
data = [
    # long term precipitation stations
    go.Scattermapbox(
        name='Precipitation records',
        lon=precip.lon.values,
        lat=precip.lat.values,
        mode='markers+text',
        marker=dict(
            size=14,
            color=mycolors.precip
        ),
        text=precip.index.values,
        textposition="top right",
        hoverinfo='none',
    ),
    # SLF stations
    go.Scattermapbox(
        name='SLF stations',
        lon=slf.lon.values,
        lat=slf.lat.values,
        mode='markers+text',
        marker=dict(
            size=14,
            color=mycolors.slf2
        ),
        text=['{}\n{:.0f} mm/y'.format(name, precip) for name, precip in zip(slf.index, slf.yearly_precip)],
        textposition="top right",
        hoverinfo='none'
    ),
    # long term temperature stations
    go.Scattermapbox(
        name='Temperature records',
        lon=temp.lon.values,
        lat=temp.lat.values,
        mode='markers+text',
        marker=dict(
            size=14,
            color=mycolors.temp
        ),
        text=temp.index.values,
        textposition="top right",
        hoverinfo='none'
    ),
    # glacier
    go.Scattermapbox(
        name='Glacier',
        lon=glacier.lon.values,
        lat=glacier.lat.values,
        mode='markers+text',
        marker=dict(
            size=14,
            color=mycolors.glacier
        ),
        text=glacier.name.values,
        hoverinfo='none'
    ),
    # HistAlp reference pixel
    go.Scattermapbox(
        name='HistAlp',
        lon=histalp.lon.values,
        lat=histalp.lat.values,
        mode='markers+text',
        marker=dict(
            size=14,
            color=mycolors.histalp
        ),
        text=histalp.name.values,
        hoverinfo='none'
    ),
]

# specify layout
layout = go.Layout(
    autosize=True,
    title='Stations',
    hovermode='closest',
    width=800,
    height=500,
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=20,
        t=40,
        pad=0,
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
        lon=histalp.lon.values[-1],
        lat=histalp.lat.values[-1]
        ),
        pitch=0,
        zoom=10.5,
        style='outdoors'
    ),
)

# create map
fig = dict(data=data, layout=layout)

# plot/show map
iplot(fig)
path = '/Users/oberrauch/Desktop/grindel_map.html'
plot(fig, path)

'file:///Users/oberrauch/work/grindelwald/notebooks/temp-plot.html'